In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from prepare import wrangle

In [22]:
train, validate, test = wrangle()

opening data from file


In [23]:
train.churn.value_counts()

0    2897
1    1046
Name: churn, dtype: int64

Our baseline prediction will be that a customer will not churn (churn = 0)

In [111]:
baseline = [0] * len(train)

In [30]:
len(baseline) == len(train)

True

In [32]:
print(f'The baseline prediction score accuracy is: {(train.churn == baseline).mean():.2%}')

The baseline prediction score accuracy is: 73.47%


In [110]:
recall_score(train[y_col], baseline)

0.0

In [6]:
train.columns

Index(['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'online_security',
       'online_backup', 'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies', 'paperless_billing', 'monthly_charges',
       'total_charges', 'churn', 'contract_type', 'internet_service_type',
       'payment_type', 'gender_male', 'multiple_lines_no_phone_service',
       'multiple_lines_yes', 'online_security_no_internet_service',
       'online_security_yes', 'online_backup_no_internet_service',
       'online_backup_yes', 'device_protection_no_internet_service',
       'device_protection_yes', 'tech_support_no_internet_service',
       'tech_support_yes', 'streaming_tv_no_internet_service',
       'streaming_tv_yes', 'streaming_movies_no_internet_service',
       'streaming_movies_yes', 'contract_type_one_year',
       'contract_type_two_year', 'internet_service_type_fiber_optic',
       'internet_service_type_non

In [7]:
X_cols_a = ['gender_male', 'online_security_no_internet_service',
       'online_security_yes', 'online_backup_no_internet_service',
       'online_backup_yes', 'device_protection_no_internet_service',
       'device_protection_yes', 'tech_support_no_internet_service',
       'tech_support_yes']
y_col = 'churn'

In [107]:
# make the decision tree object
dt_1 = DecisionTreeClassifier()
# fit the data to the dt object
dt_1.fit(train[X_cols_a], train[y_col])
# predict with the dt object
dt_1_preds = dt_1.predict(train[X_cols_a])

print(classification_report(train[y_col], dt_1_preds))
print()
print(confusion_matrix(train[y_col], dt_1_preds))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84      2897
           1       0.56      0.38      0.45      1046

    accuracy                           0.76      3943
   macro avg       0.68      0.64      0.65      3943
weighted avg       0.74      0.76      0.74      3943


[[2588  309]
 [ 650  396]]


In [106]:
# create the random forest classifier object
rf_1 = RandomForestClassifier()
# fit the rf object
rf_1.fit(train[X_cols_a], train[y_col])
# predict with the rf object
rf_1_preds = rf_1.predict(train[X_cols_a])

print(classification_report(train[y_col], rf_1_preds))
print()
print(confusion_matrix(train[y_col], rf_1_preds))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84      2897
           1       0.56      0.38      0.45      1046

    accuracy                           0.76      3943
   macro avg       0.68      0.64      0.65      3943
weighted avg       0.74      0.76      0.74      3943


[[2588  309]
 [ 650  396]]


In [104]:
# create the K_Nearest_Neighbors object
knn_1 = KNeighborsClassifier()
# fit the KNN object
knn_1.fit(train[X_cols_a], train[y_col])
# predict with the knn object
knn_1_preds = knn_1.predict(train[X_cols_a])

print(classification_report(train[y_col], knn_1_preds))
print()
print(confusion_matrix(train[y_col], knn_1_preds))

              precision    recall  f1-score   support

           0       0.76      0.90      0.82      2897
           1       0.41      0.19      0.26      1046

    accuracy                           0.71      3943
   macro avg       0.58      0.55      0.54      3943
weighted avg       0.66      0.71      0.67      3943


[[2614  283]
 [ 846  200]]


/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [105]:
# create the logistic regression object
lr_1 = LogisticRegression()
# fit the lr object
lr_1.fit(train[X_cols_a], train[y_col])
# predict with the lr object
lr_1_preds = lr_1.predict(train[X_cols_a])

print(classification_report(train[y_col], lr_1_preds))
print()
print(confusion_matrix(train[y_col], lr_1_preds))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84      2897
           1       0.56      0.38      0.45      1046

    accuracy                           0.76      3943
   macro avg       0.68      0.64      0.65      3943
weighted avg       0.74      0.76      0.74      3943


[[2588  309]
 [ 650  396]]


In [139]:
train['security_and_backup'] = ((train.online_backup == 'Yes') 
                                   & (train.online_security == 'Yes'))
validate['security_and_backup'] = ((validate.online_backup == 'Yes') 
                                   & (validate.online_security == 'Yes'))
test['security_and_backup'] = ((test.online_backup == 'Yes') 
                                   & (test.online_security == 'Yes'))

In [140]:
train['protection_and_tech'] = ((train.device_protection_yes == 1) 
                        & train.tech_support_yes == 1)
validate['protection_and_tech'] = ((validate.device_protection_yes == 1) 
                        & validate.tech_support_yes == 1)
test['protection_and_tech'] = ((test.device_protection_yes == 1) 
                        & test.tech_support_yes == 1)

In [50]:
X_cols_b = ['gender_male', 'online_security_no_internet_service',
       'online_security_yes', 'online_backup_no_internet_service',
       'online_backup_yes', 'device_protection_no_internet_service',
       'device_protection_yes', 'tech_support_no_internet_service',
       'tech_support_yes', 'security_and_backup', 'protection_and_tech', 
       'internet_service_type_fiber_optic', 'internet_service_type_none']

In [51]:
# make the decision tree object
dt_2 = DecisionTreeClassifier()
# fit the data to the dt object
dt_2.fit(train[X_cols_b], train[y_col])
# predict with the dt object
dt_2_preds = dt_2.predict(train[X_cols_b])

In [52]:
print(classification_report(train[y_col], dt_2_preds))
print()
print(confusion_matrix(train[y_col], dt_2_preds))

              precision    recall  f1-score   support

           0       0.80      0.93      0.86      2897
           1       0.63      0.34      0.44      1046

    accuracy                           0.77      3943
   macro avg       0.71      0.63      0.65      3943
weighted avg       0.75      0.77      0.75      3943


[[2689  208]
 [ 690  356]]


In [53]:
# create the random forest classifier object
rf_2 = RandomForestClassifier()
# fit the rf object
rf_2.fit(train[X_cols_b], train[y_col])
# predict with the rf object
rf_2_preds = rf_2.predict(train[X_cols_b])

In [54]:
print(classification_report(train[y_col], rf_2_preds))
print()
print(confusion_matrix(train[y_col], rf_2_preds))

              precision    recall  f1-score   support

           0       0.80      0.93      0.86      2897
           1       0.63      0.34      0.44      1046

    accuracy                           0.77      3943
   macro avg       0.71      0.63      0.65      3943
weighted avg       0.75      0.77      0.75      3943


[[2689  208]
 [ 690  356]]


In [55]:
# create the K_Nearest_Neighbors object
knn_2 = KNeighborsClassifier()
# fit the KNN object
knn_2.fit(train[X_cols_b], train[y_col])
# predict with the knn object
knn_2_preds = knn_2.predict(train[X_cols_b])

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [56]:
print(classification_report(train[y_col], knn_2_preds))
print()
print(confusion_matrix(train[y_col], knn_2_preds))

              precision    recall  f1-score   support

           0       0.79      0.87      0.83      2897
           1       0.49      0.36      0.42      1046

    accuracy                           0.73      3943
   macro avg       0.64      0.61      0.62      3943
weighted avg       0.71      0.73      0.72      3943


[[2514  383]
 [ 671  375]]


In [57]:
# create the logistic regression object
lr_2 = LogisticRegression()
# fit the lr object
lr_2.fit(train[X_cols_b], train[y_col])
# predict with the lr object
lr_2_preds = lr_2.predict(train[X_cols_b])

In [58]:
print(classification_report(train[y_col], lr_2_preds))
print()
print(confusion_matrix(train[y_col], lr_2_preds))

              precision    recall  f1-score   support

           0       0.80      0.91      0.85      2897
           1       0.61      0.38      0.47      1046

    accuracy                           0.77      3943
   macro avg       0.71      0.65      0.66      3943
weighted avg       0.75      0.77      0.75      3943


[[2646  251]
 [ 647  399]]


In [86]:
X_cols_c = ['gender_male', 'streaming_tv_no_internet_service',
       'streaming_tv_yes', 'streaming_movies_no_internet_service',
       'streaming_movies_yes', 'internet_service_type_fiber_optic',
       'internet_service_type_none']

In [87]:
# make the decision tree object
dt_3 = DecisionTreeClassifier()
# fit the data to the dt object
dt_3.fit(train[X_cols_c], train[y_col])
# predict with the dt object
dt_3_preds = dt_3.predict(train[X_cols_c])

In [88]:
print(classification_report(train[y_col], dt_3_preds))
print()
print(confusion_matrix(train[y_col], dt_3_preds))

              precision    recall  f1-score   support

           0       0.75      0.96      0.84      2897
           1       0.51      0.12      0.20      1046

    accuracy                           0.74      3943
   macro avg       0.63      0.54      0.52      3943
weighted avg       0.69      0.74      0.67      3943


[[2772  125]
 [ 917  129]]


In [89]:
# create the random forest classifier object
rf_3 = RandomForestClassifier()
# fit the rf object
rf_3.fit(train[X_cols_c], train[y_col])
# predict with the rf object
rf_3_preds = rf_3.predict(train[X_cols_c])

In [90]:
print(classification_report(train[y_col], rf_3_preds))
print()
print(confusion_matrix(train[y_col], rf_3_preds))

              precision    recall  f1-score   support

           0       0.75      0.96      0.84      2897
           1       0.51      0.12      0.20      1046

    accuracy                           0.74      3943
   macro avg       0.63      0.54      0.52      3943
weighted avg       0.69      0.74      0.67      3943


[[2772  125]
 [ 917  129]]


In [91]:
# create the K_Nearest_Neighbors object
knn_3 = KNeighborsClassifier()
# fit the KNN object
knn_3.fit(train[X_cols_c], train[y_col])
# predict with the knn object
knn_3_preds = knn_3.predict(train[X_cols_c])

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [92]:
print(classification_report(train[y_col], knn_3_preds))
print()
print(confusion_matrix(train[y_col], knn_3_preds))

              precision    recall  f1-score   support

           0       0.77      0.88      0.82      2897
           1       0.44      0.25      0.32      1046

    accuracy                           0.72      3943
   macro avg       0.60      0.57      0.57      3943
weighted avg       0.68      0.72      0.69      3943


[[2556  341]
 [ 782  264]]


In [93]:
# create the logistic regression object
lr_3 = LogisticRegression()
# fit the lr object
lr_3.fit(train[X_cols_c], train[y_col])
# predict with the lr object
lr_3_preds = lr_3.predict(train[X_cols_c])

In [94]:
print(classification_report(train[y_col], lr_3_preds))
print()
print(confusion_matrix(train[y_col], lr_3_preds))

              precision    recall  f1-score   support

           0       0.77      0.91      0.83      2897
           1       0.50      0.24      0.32      1046

    accuracy                           0.73      3943
   macro avg       0.63      0.58      0.58      3943
weighted avg       0.70      0.73      0.70      3943


[[2642  255]
 [ 796  250]]


In [95]:
X_cols_d = ['gender_male', 'multiple_lines_no_phone_service',
       'multiple_lines_yes', 'online_security_no_internet_service',
       'online_security_yes', 'online_backup_no_internet_service',
       'online_backup_yes', 'device_protection_no_internet_service',
       'device_protection_yes', 'tech_support_no_internet_service',
       'tech_support_yes', 'streaming_tv_no_internet_service',
       'streaming_tv_yes', 'streaming_movies_no_internet_service',
       'streaming_movies_yes', 'contract_type_one_year',
       'contract_type_two_year', 'internet_service_type_fiber_optic',
       'internet_service_type_none', 'payment_type_credit_card_automatic',
       'payment_type_electronic_check', 'payment_type_mailed_check']

In [96]:
# make the decision tree object
dt_4 = DecisionTreeClassifier()
# fit the data to the dt object
dt_4.fit(train[X_cols_d], train[y_col])
# predict with the dt object
dt_4_preds = dt_4.predict(train[X_cols_d])

In [97]:
print(classification_report(train[y_col], dt_4_preds))
print()
print(confusion_matrix(train[y_col], dt_4_preds))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91      2897
           1       0.77      0.70      0.73      1046

    accuracy                           0.87      3943
   macro avg       0.83      0.81      0.82      3943
weighted avg       0.86      0.87      0.86      3943


[[2682  215]
 [ 316  730]]


In [108]:
# create the random forest classifier object
rf_4 = RandomForestClassifier()
# fit the rf object
rf_4.fit(train[X_cols_d], train[y_col])
# predict with the rf object
rf_4_preds = rf_4.predict(train[X_cols_d])

print(classification_report(train[y_col], rf_4_preds))
print()
print(confusion_matrix(train[y_col], rf_4_preds))

              precision    recall  f1-score   support

           0       0.91      0.90      0.91      2897
           1       0.74      0.76      0.75      1046

    accuracy                           0.87      3943
   macro avg       0.83      0.83      0.83      3943
weighted avg       0.87      0.87      0.87      3943


[[2617  280]
 [ 251  795]]


In [100]:
# create the K_Nearest_Neighbors object
knn_4 = KNeighborsClassifier()
# fit the KNN object
knn_4.fit(train[X_cols_d], train[y_col])
# predict with the knn object
knn_4_preds = knn_4.predict(train[X_cols_d])

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [101]:
print(classification_report(train[y_col], knn_4_preds))
print()
print(confusion_matrix(train[y_col], knn_4_preds))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87      2897
           1       0.64      0.59      0.61      1046

    accuracy                           0.80      3943
   macro avg       0.75      0.73      0.74      3943
weighted avg       0.80      0.80      0.80      3943


[[2554  343]
 [ 433  613]]


In [102]:
# create the logistic regression object
lr_4 = LogisticRegression()
# fit the lr object
lr_4.fit(train[X_cols_d], train[y_col])
# predict with the lr object
lr_4_preds = lr_4.predict(train[X_cols_d])

In [103]:
print(classification_report(train[y_col], lr_4_preds))
print()
print(confusion_matrix(train[y_col], lr_4_preds))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      2897
           1       0.61      0.52      0.56      1046

    accuracy                           0.78      3943
   macro avg       0.72      0.70      0.71      3943
weighted avg       0.77      0.78      0.78      3943


[[2547  350]
 [ 503  543]]


In [109]:
rf_4_preds_val = rf_4.predict(validate[X_cols_d])

print(classification_report(validate[y_col], rf_4_preds_val))
print()
print(confusion_matrix(validate[y_col], rf_4_preds_val))

              precision    recall  f1-score   support

           0       0.83      0.84      0.83      1242
           1       0.54      0.52      0.53       449

    accuracy                           0.76      1691
   macro avg       0.68      0.68      0.68      1691
weighted avg       0.75      0.76      0.75      1691


[[1043  199]
 [ 215  234]]


In [115]:
def create_classification_models(X_cols, y_col):
    # make the decision tree object
    dt = DecisionTreeClassifier()
    # fit the data to the dt object
    dt.fit(train[X_cols], train[y_col])
    # predict with the dt object
    dt_preds = dt.predict(train[X_cols])
    dt_val_preds = dt.predict(validate[X_cols])
    print('decision tree')
    print(classification_report(train[y_col], dt_preds))
    print()
    print(confusion_matrix(train[y_col], dt_preds))
    print('----------validation data----------')
    print(classification_report(validate[y_col], dt_val_preds))
    print()
    print(confusion_matrix(validate[y_col], dt_val_preds))
    print('\n\n')
    
    # create the random forest classifier object
    rf = RandomForestClassifier()
    # fit the rf object
    rf.fit(train[X_cols], train[y_col])
    # predict with the rf object
    rf_preds = rf.predict(train[X_cols])
    rf_val_preds = rf.predict(validate[X_cols])
    print('random forest')
    print(classification_report(train[y_col], rf_preds))
    print()
    print(confusion_matrix(train[y_col], rf_preds))
    print('----------validation data----------')
    print(classification_report(validate[y_col], rf_val_preds))
    print()
    print(confusion_matrix(validate[y_col], rf_val_preds))
    print('\n\n')
    
    # create the K_Nearest_Neighbors object
    knn = KNeighborsClassifier()
    # fit the KNN object
    knn.fit(train[X_cols], train[y_col])
    # predict with the knn object
    knn_preds = knn.predict(train[X_cols])
    knn_val_preds = knn.predict(validate[X_cols])
    print('k nearest neighbors')
    print(classification_report(train[y_col], knn_preds))
    print()
    print(confusion_matrix(train[y_col], knn_preds))
    print('----------validation data----------')
    print(classification_report(validate[y_col], knn_val_preds))
    print()
    print(confusion_matrix(validate[y_col], knn_val_preds))
    print('\n\n')
    
    # create the logistic regression object
    lr = LogisticRegression()
    # fit the lr object
    lr.fit(train[X_cols], train[y_col])
    # predict with the lr object
    lr_preds = lr.predict(train[X_cols])
    lr_val_preds = lr.predict(validate[X_cols])
    print('logistic regression')
    print(classification_report(train[y_col], lr_preds))
    print()
    print(confusion_matrix(train[y_col], lr_preds))
    print('----------validation data----------')
    print(classification_report(validate[y_col], lr_val_preds))
    print()
    print(confusion_matrix(validate[y_col], lr_val_preds))

In [116]:
create_classification_models(X_cols_d, y_col)

decision tree
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      2897
           1       0.77      0.70      0.73      1046

    accuracy                           0.87      3943
   macro avg       0.83      0.81      0.82      3943
weighted avg       0.86      0.87      0.86      3943


[[2682  215]
 [ 316  730]]
----------validation data----------
              precision    recall  f1-score   support

           0       0.82      0.83      0.83      1242
           1       0.52      0.50      0.51       449

    accuracy                           0.74      1691
   macro avg       0.67      0.67      0.67      1691
weighted avg       0.74      0.74      0.74      1691


[[1033  209]
 [ 225  224]]



random forest
              precision    recall  f1-score   support

           0       0.91      0.90      0.91      2897
           1       0.74      0.76      0.75      1046

    accuracy                           0.87      3943
   ma

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

k nearest neighbors
              precision    recall  f1-score   support

           0       0.86      0.88      0.87      2897
           1       0.64      0.59      0.61      1046

    accuracy                           0.80      3943
   macro avg       0.75      0.73      0.74      3943
weighted avg       0.80      0.80      0.80      3943


[[2554  343]
 [ 433  613]]
----------validation data----------
              precision    recall  f1-score   support

           0       0.82      0.84      0.83      1242
           1       0.52      0.49      0.50       449

    accuracy                           0.75      1691
   macro avg       0.67      0.66      0.67      1691
weighted avg       0.74      0.75      0.74      1691


[[1041  201]
 [ 230  219]]



logistic regression
              precision    recall  f1-score   support

           0       0.84      0.88      0.86      2897
           1       0.61      0.52      0.56      1046

    accuracy                           0.78    

In [123]:
X_cols_e = ['senior_citizen', 'partner', 'dependents',
       'gender_male', 'multiple_lines_no_phone_service',
       'multiple_lines_yes', 'online_security_yes', 'online_backup_yes',
       'device_protection_yes', 'tech_support_yes', 'streaming_tv_yes',
       'streaming_movies_yes', 'contract_type_one_year',
       'contract_type_two_year', 'internet_service_type_fiber_optic',
       'internet_service_type_none', 'payment_type_credit_card_automatic',
       'payment_type_electronic_check', 'payment_type_mailed_check']

In [124]:
create_classification_models(X_cols_e, y_col)

decision tree
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      2897
           1       0.89      0.78      0.83      1046

    accuracy                           0.92      3943
   macro avg       0.91      0.87      0.89      3943
weighted avg       0.92      0.92      0.91      3943


[[2795  102]
 [ 227  819]]
----------validation data----------
              precision    recall  f1-score   support

           0       0.81      0.82      0.82      1242
           1       0.49      0.46      0.47       449

    accuracy                           0.73      1691
   macro avg       0.65      0.64      0.65      1691
weighted avg       0.72      0.73      0.73      1691


[[1022  220]
 [ 241  208]]



random forest
              precision    recall  f1-score   support

           0       0.94      0.95      0.94      2897
           1       0.86      0.82      0.84      1046

    accuracy                           0.92      3943
   ma

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

k nearest neighbors
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      2897
           1       0.69      0.61      0.65      1046

    accuracy                           0.83      3943
   macro avg       0.78      0.76      0.77      3943
weighted avg       0.82      0.83      0.82      3943


[[2617  280]
 [ 409  637]]
----------validation data----------
              precision    recall  f1-score   support

           0       0.81      0.83      0.82      1242
           1       0.51      0.47      0.49       449

    accuracy                           0.74      1691
   macro avg       0.66      0.65      0.66      1691
weighted avg       0.73      0.74      0.73      1691


[[1036  206]
 [ 238  211]]



logistic regression
              precision    recall  f1-score   support

           0       0.84      0.89      0.86      2897
           1       0.62      0.52      0.57      1046

    accuracy                           0.79    

In [125]:
create_classification_models(X_cols_d, y_col)

decision tree
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      2897
           1       0.77      0.70      0.73      1046

    accuracy                           0.87      3943
   macro avg       0.83      0.81      0.82      3943
weighted avg       0.86      0.87      0.86      3943


[[2682  215]
 [ 316  730]]
----------validation data----------
              precision    recall  f1-score   support

           0       0.82      0.83      0.82      1242
           1       0.51      0.50      0.50       449

    accuracy                           0.74      1691
   macro avg       0.67      0.66      0.66      1691
weighted avg       0.74      0.74      0.74      1691


[[1027  215]
 [ 225  224]]



random forest
              precision    recall  f1-score   support

           0       0.91      0.91      0.91      2897
           1       0.75      0.75      0.75      1046

    accuracy                           0.87      3943
   ma

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

k nearest neighbors
              precision    recall  f1-score   support

           0       0.86      0.88      0.87      2897
           1       0.64      0.59      0.61      1046

    accuracy                           0.80      3943
   macro avg       0.75      0.73      0.74      3943
weighted avg       0.80      0.80      0.80      3943


[[2554  343]
 [ 433  613]]
----------validation data----------
              precision    recall  f1-score   support

           0       0.82      0.84      0.83      1242
           1       0.52      0.49      0.50       449

    accuracy                           0.75      1691
   macro avg       0.67      0.66      0.67      1691
weighted avg       0.74      0.75      0.74      1691


[[1041  201]
 [ 230  219]]



logistic regression
              precision    recall  f1-score   support

           0       0.84      0.88      0.86      2897
           1       0.61      0.52      0.56      1046

    accuracy                           0.78    

In [145]:
X_cols_f = ['gender_male', 'multiple_lines_no_phone_service',
            'senior_citizen', 'partner',
       'multiple_lines_yes',
       'online_security_yes',
       'online_backup_yes',
       'device_protection_yes',
       'tech_support_yes',
       'streaming_tv_yes',
       'streaming_movies_yes', 'contract_type_one_year',
       'contract_type_two_year', 'internet_service_type_fiber_optic',
       'internet_service_type_none', 'payment_type_credit_card_automatic',
       'payment_type_electronic_check', 'payment_type_mailed_check']

In [146]:
create_classification_models(X_cols_f, y_col)

decision tree
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      2897
           1       0.88      0.74      0.81      1046

    accuracy                           0.91      3943
   macro avg       0.90      0.85      0.87      3943
weighted avg       0.90      0.91      0.90      3943


[[2794  103]
 [ 267  779]]
----------validation data----------
              precision    recall  f1-score   support

           0       0.80      0.83      0.81      1242
           1       0.47      0.41      0.44       449

    accuracy                           0.72      1691
   macro avg       0.63      0.62      0.63      1691
weighted avg       0.71      0.72      0.71      1691


[[1033  209]
 [ 263  186]]



random forest
              precision    recall  f1-score   support

           0       0.93      0.94      0.94      2897
           1       0.84      0.80      0.82      1046

    accuracy                           0.91      3943
   ma

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

k nearest neighbors
              precision    recall  f1-score   support

           0       0.86      0.89      0.88      2897
           1       0.66      0.61      0.64      1046

    accuracy                           0.81      3943
   macro avg       0.76      0.75      0.76      3943
weighted avg       0.81      0.81      0.81      3943


[[2569  328]
 [ 405  641]]
----------validation data----------
              precision    recall  f1-score   support

           0       0.81      0.83      0.82      1242
           1       0.49      0.47      0.48       449

    accuracy                           0.73      1691
   macro avg       0.65      0.65      0.65      1691
weighted avg       0.73      0.73      0.73      1691


[[1027  215]
 [ 240  209]]



logistic regression
              precision    recall  f1-score   support

           0       0.84      0.89      0.86      2897
           1       0.62      0.52      0.57      1046

    accuracy                           0.79    

In [159]:
X_cols_g = ['gender_male', 'multiple_lines_no_phone_service',
       'multiple_lines_yes',
       'online_security_yes',
       'online_backup_yes',
       'device_protection_yes', 
       'tech_support_yes',
       'streaming_tv_yes',
       'streaming_movies_yes', 'contract_type_one_year',
       'contract_type_two_year', 'internet_service_type_fiber_optic',
       'internet_service_type_none', 'payment_type_credit_card_automatic',
       'payment_type_electronic_check', 'payment_type_mailed_check']

In [165]:
create_classification_models(X_cols_g, y_col)

decision tree
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      2897
           1       0.77      0.70      0.73      1046

    accuracy                           0.87      3943
   macro avg       0.83      0.81      0.82      3943
weighted avg       0.86      0.87      0.86      3943


[[2682  215]
 [ 316  730]]
----------validation data----------
              precision    recall  f1-score   support

           0       0.82      0.83      0.83      1242
           1       0.52      0.50      0.51       449

    accuracy                           0.74      1691
   macro avg       0.67      0.66      0.67      1691
weighted avg       0.74      0.74      0.74      1691


[[1033  209]
 [ 226  223]]



random forest
              precision    recall  f1-score   support

           0       0.91      0.90      0.91      2897
           1       0.74      0.76      0.75      1046

    accuracy                           0.87      3943
   ma

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

k nearest neighbors
              precision    recall  f1-score   support

           0       0.85      0.89      0.87      2897
           1       0.65      0.55      0.60      1046

    accuracy                           0.80      3943
   macro avg       0.75      0.72      0.73      3943
weighted avg       0.79      0.80      0.80      3943


[[2581  316]
 [ 466  580]]
----------validation data----------
              precision    recall  f1-score   support

           0       0.82      0.85      0.83      1242
           1       0.53      0.49      0.51       449

    accuracy                           0.75      1691
   macro avg       0.68      0.67      0.67      1691
weighted avg       0.74      0.75      0.75      1691


[[1050  192]
 [ 230  219]]



logistic regression
              precision    recall  f1-score   support

           0       0.84      0.88      0.86      2897
           1       0.61      0.52      0.56      1046

    accuracy                           0.78    

In [160]:
X_list = [X_cols_a, X_cols_b, X_cols_c, X_cols_d, X_cols_e, X_cols_f, X_cols_g]
for X in X_list:
    lr = LogisticRegression()
    lr.fit(train[X], train[y_col])
    pred = lr.predict(train[X])
    val_pred = lr.predict(validate[X])
    print(f'train recall score is: {recall_score(train[y_col], pred):.2%}')
    print(f'val recall score is: {recall_score(validate[y_col], val_pred):.2%}')

train recall score is: 37.86%
val recall score is: 39.20%
train recall score is: 38.15%
val recall score is: 36.30%
train recall score is: 23.90%
val recall score is: 22.72%
train recall score is: 51.91%
val recall score is: 55.23%
train recall score is: 52.29%
val recall score is: 52.12%
train recall score is: 52.20%
val recall score is: 51.22%
train recall score is: 51.91%
val recall score is: 55.23%


In [163]:
test_pred = lr.predict(test[X_cols_g])
print(f'test recall score is: {recall_score(test[y_col], test_pred):.2%}')
print(classification_report(test[y_col], test_pred))

test recall score is: 52.41%
              precision    recall  f1-score   support

           0       0.84      0.89      0.87      1035
           1       0.64      0.52      0.58       374

    accuracy                           0.80      1409
   macro avg       0.74      0.71      0.72      1409
weighted avg       0.79      0.80      0.79      1409

